# Hvordan lage spørring som gir både kommunenavn og kommunenummer (id) fra JSON-stat

### Importerer biblioteker
Bruker [pyjstat](https://pypi.org/project/pyjstat/) og pandas. Pandas lastes som del av pyjstat

In [1]:
from pyjstat import pyjstat
import requests

URL med tabellens metadata, som vi skal poste spørringen mot

In [2]:
POST_URL = 'https://data.ssb.no/api/v0/no/table/07459'

Spørring på en linje - kan også tas fra konsoll.
0, 1 og 2-åringer i Oslo, Bergen Trondheim i 2018 og 2019

In [3]:
json_q = {"query":[{"code":"Region","selection":{"filter":"item","values":["0301","1103","5001"]}}, {"code":"Kjonn","selection":{"filter":"item","values":["1","2"]}},{"code":"Alder","selection":{"filter":"item","values":["000","001","002"]}}, {"code":"ContentsCode","selection":{"filter":"item","values":["Personer1"]}},{"code":"Tid","selection":{"filter":"item","values":["2018","2019","2020"]}}], "response":{"format":"json-stat2"}}

Poster spørringen

In [4]:
res = requests.post(POST_URL, json=json_q)

### Leser JSON-stat resultatet med biblioteket pyjstat
Lagrer det som datasett ds.

In [5]:
ds = pyjstat.Dataset.read(res.text)

In [6]:
type(ds)

pyjstat.pyjstat.Dataset

Se datasettet ds

In [7]:
ds

Dataset([('class', 'dataset'),
         ('label',
          '07459: Befolkning, etter region, kjønn, alder, statistikkvariabel og år'),
         ('source', 'Statistisk sentralbyrå'),
         ('updated', '2020-02-27T07:00:00Z'),
         ('id', ['Region', 'Kjonn', 'Alder', 'ContentsCode', 'Tid']),
         ('size', [3, 2, 3, 1, 3]),
         ('dimension',
          OrderedDict([('Region',
                        OrderedDict([('label', 'region'),
                                     ('category',
                                      OrderedDict([('index',
                                                    OrderedDict([('0301', 0),
                                                                 ('1103', 1),
                                                                 ('5001',
                                                                  2)])),
                                                   ('label',
                                                    OrderedDict([('0301',
 

### Henter noen overordnete metadata fra JSON-stat datasettet

In [8]:
tittel = ds['label']
print(tittel)

07459: Befolkning, etter region, kjønn, alder, statistikkvariabel og år


Sist Oppdatert som GMT

In [9]:
last_update = ds['updated']
print(last_update)

2020-02-27T07:00:00Z


Henter kilde

In [10]:
source = ds['source']
print(source)

Statistisk sentralbyrå


### Role gir noen snarveier, 
*men det er en liten bug i Statistikkbanken som ikke får med role - geo : 'Region' for denne tabellen*

In [11]:
ds_roles = ds['role']
print(ds_roles)

OrderedDict([('time', ['Tid']), ('metric', ['ContentsCode'])])


### Lager dataframes
Vi må lage to dataframes, en med tekst og en med ID. Pyjstat tillater bare enten/eller

In [12]:
befolkning = ds.write('dataframe')
befolkning_id = ds.write('dataframe', naming='id')

Lager en ny kolonne med både kommunenummer og navn slått sammen

In [13]:
befolkning['kommune'] = befolkning_id['Region'] + ' ' + befolkning['region']

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


Lager en ny dataframe som bare har med kolonnene vi ønsker, i ny rekkefølge. Merk doble [[ ]]

In [14]:
befolkning_ny = befolkning[['kommune', 'kjønn', 'alder', 'år', 'value']]

In [15]:
print(befolkning_ny)

              kommune    kjønn alder    år  value
0   0301 Oslo kommune     Menn  0 år  2018   4779
1   0301 Oslo kommune     Menn  0 år  2019   4523
2   0301 Oslo kommune     Menn  0 år  2020   4647
3   0301 Oslo kommune     Menn  1 år  2018   4709
4   0301 Oslo kommune     Menn  1 år  2019   4507
5   0301 Oslo kommune     Menn  1 år  2020   4283
6   0301 Oslo kommune     Menn  2 år  2018   4498
7   0301 Oslo kommune     Menn  2 år  2019   4463
8   0301 Oslo kommune     Menn  2 år  2020   4280
9   0301 Oslo kommune  Kvinner  0 år  2018   4456
10  0301 Oslo kommune  Kvinner  0 år  2019   4479
11  0301 Oslo kommune  Kvinner  0 år  2020   4454
12  0301 Oslo kommune  Kvinner  1 år  2018   4362
13  0301 Oslo kommune  Kvinner  1 år  2019   4186
14  0301 Oslo kommune  Kvinner  1 år  2020   4159
15  0301 Oslo kommune  Kvinner  2 år  2018   4015
16  0301 Oslo kommune  Kvinner  2 år  2019   4144
17  0301 Oslo kommune  Kvinner  2 år  2020   3975
18     1103 Stavanger     Menn  0 år  2018    835


Merk at denne spørringen i dette eksempelet bruker rene kommunenummer, og ikke K-koder for sammenslåtte kommuner i tidsserier. 

En spørring som inkluderer tidsserier for kommuner som er berørt av kommunereformen, ville vært mer kompleks.